<a href="https://colab.research.google.com/github/gordeli/NLP_EDHEC/blob/main/colab/05_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Natural Language Processing @ EDHEC, 2022

# Part 5: Final

[<- Previous: Content Analysis](https://colab.research.google.com/github/gordeli/NLP_EDHEC/blob/main/colab/04_Content_Analysis.ipynb)

Dates: January 31 - February 11, 2022

Facilitator: [Ivan Gordeliy](https://www.linkedin.com/in/gordeli/)

---



## Initial Setup

- **Run "Setup" below first.**

    - This will load libraries and download some resources that we'll use throughout the tutorial.

    - You will see a message reading "Done with setup!" when this process completes.
    

In [ ]:
#@title Setup (click the "run" button to the left) {display-mode: "form"}

## Setup ##

# imports

# built-in Python libraries
# -------------------------
import collections
import re
import string
import warnings
warnings.filterwarnings('ignore')

# 3rd party libraries
# -------------------

# Natural Language Toolkit (https://www.nltk.org/)
import nltk

# download punctuation related NLTK functions
# (needed for sent_tokenize())
nltk.download('punkt')
# download NLKT part-of-speech tagger
# (needed for pos_tag())
nltk.download('averaged_perceptron_tagger')
# download wordnet
# (needed for lemmatization)
nltk.download('wordnet')
# download stopword lists
# (needed for stopword removal)
nltk.download('stopwords')
# dictionary of English words
nltk.download('words')

# numpy: matrix library for Python
import numpy as np

!pip install -U gensim

# Gensim for topic modeling
import gensim
# for loading data
import sklearn.datasets
# for LDA visualization
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

# for uploading data files
from google.colab import files

# downloading values lexicon
!wget https://raw.githubusercontent.com/steve-wilson/values_lexicon/master/lexicon_1_0/values_lexicon.txt
!wget https://raw.githubusercontent.com/steve-wilson/values_lexicon/master/sample_data/subreddits/christian_500.txt
!wget https://raw.githubusercontent.com/steve-wilson/values_lexicon/master/sample_data/subreddits/business_500.txt
!wget https://raw.githubusercontent.com/steve-wilson/values_lexicon/master/sample_data/subreddits/college_500.txt

def text_to_lemma_frequencies(text, remove_stop_words=True):
    
    # split document into sentences
    sentences = nltk.sent_tokenize(text)
    
    # create a place to store (word, pos_tag) tuples
    words_and_pos_tags = []
    
    # get all words and pos tags
    for sentence in sentences:
        words_and_pos_tags += nltk.pos_tag(nltk.word_tokenize(sentence))
        
    # load the lemmatizer
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    # lemmatize the words
    lemmas = [lemmatizer.lemmatize(word,lookup_pos(pos)) for \
              (word,pos) in words_and_pos_tags]
    
    # convert to lowercase
    lowercase_lemmas = [lemma.lower() for lemma in lemmas]
    
    # load the stopword list for English
    stop_words = set([])
    if remove_stop_words:
        stop_words = set(nltk.corpus.stopwords.words('english'))
    
    # add punctuation to the set of things to remove
    all_removal_tokens = stop_words | set(string.punctuation)
    
    # bonus: also add some custom double-quote tokens to this set
    all_removal_tokens |= set(["''","``"])
    
    # only get lemmas that aren't in these lists
    content_lemmas = [lemma for lemma in lowercase_lemmas \
                      if lemma not in all_removal_tokens and \
                      re.match(r"^\w+$",lemma)]
    
    # return the frequency distribution object
    return nltk.probability.FreqDist(content_lemmas)
    
def docs2matrix(document_list):
    
    # use the vocab2index idea from before
    vocab2index = {}
    
    # load the stopword list for English
    stop_words = set(nltk.corpus.stopwords.words('english'))
    stop_words |= set(['from', 'subject', 're', 'edu', 'use'])
    
    # add punctuation to the set of things to remove
    all_removal_tokens = stop_words | set(string.punctuation)
    
    # bonus: also add some custom double-quote tokens to this set
    all_removal_tokens |= set(["''","``"])
    
    vocab2index = {}
    latest_index = 0

    lfs = []
    # this should be a nice starting point
    for doc in document_list:
        lf = text_to_lemma_frequencies(doc,all_removal_tokens)
        for token in lf.keys():
            if token not in vocab2index:
                vocab2index[token] = latest_index
                latest_index += 1
                
        lfs.append(lf)
    
    # create the zeros matrix
    corpus_matrix = np.zeros((len(lfs), len(vocab2index)))
    
    for row, lf in enumerate(lfs):
        for token, frequency in lf.items():
            column = vocab2index[token]
            corpus_matrix[row][column] = frequency
    
    return corpus_matrix, vocab2index

    
# Lemmatization -- redefining this here to make
# code block more self-contained
def lookup_pos(pos):
    pos_first_char = pos[0].lower()
    if pos_first_char in 'nv':
        return pos_first_char
    else:
        return 'n'


            
print()
print("Done with setup!")
print("If you'd like, you can click the (X) button to the left to clear this output.")

### Topic Modeling

- Now that we have some real data, what are some ways that we can explore what's in it?
    - How can we answer the basic question: *What are people talking about in this corpus?*

- Load a corpus matrix, like the ones we created earlier, into gensim's corpus object:

In [ ]:
# this time, let's load all documents in the 20news dataset from these categories
categories = ['soc.religion.christian', 'rec.autos', 'talk.politics.misc', \
              'rec.sport.baseball', 'comp.sys.ibm.pc.hardware']
newsgroups_train_all = sklearn.datasets.fetch_20newsgroups(subset='train', \
                                              categories=categories).data
# using the function we wrote before, but modified to also return the vocab2index
corpus_matrix, word2id = docs2matrix(newsgroups_train_all)
# reverse this dictionary
id2word = {v:k for k,v in word2id.items()}

corpus = gensim.matutils.Dense2Corpus(corpus_matrix, documents_columns=False)
print("Loaded",len(corpus),"documents into a Gensim corpus.")

- Given this, we can run LDA right out of the box:

In [ ]:
# As of July 2019, gensim calls a deprecated numpy function and gives lots of warning messages
# Let's supress these.
warnings.filterwarnings('ignore')

# run LDA on our corpus, using out dictionary (k=6)
lda = gensim.models.LdaModel(corpus, id2word=id2word, num_topics=6)
lda.print_topics()

- There is still quite a bit of noise in this list because the documents are full of very common words like "write", "subject", and "from".
- One common approach is to remove the most (and possibly least) common words before running LDA.

In [ ]:
total_counts = np.sum(corpus_matrix, axis=0)
sorted_words = sorted( zip( range(len(total_counts)) ,total_counts), \
                       key=lambda x:x[1], reverse=True )
N = 100
M = 50
top_N_ids = [item[0] for item in sorted_words[:N]]
appears_less_than_M_times = [item[0] for item in sorted_words if item[1] < M]
vocab_dense = [id2word[idx] for idx in range(len(id2word))]

print("Top words to remove:", ' '.join([id2word[idx] for idx in top_N_ids]))

remove_indexes = top_N_ids+appears_less_than_M_times
corpus_matrix_filtered = np.delete(corpus_matrix,remove_indexes,1)

for index in sorted(remove_indexes, reverse=True):
    del vocab_dense[index]

id2word_filtered = {}
word2id_filtered = {}

for i,word in enumerate(vocab_dense):
    id2word_filtered[i] = word
    word2id_filtered[word] = i
    
corpus_filtered = gensim.matutils.Dense2Corpus(corpus_matrix_filtered, documents_columns=False)

print("Original matrix shape:",corpus_matrix.shape)
print("New matrix shape:",corpus_matrix_filtered.shape)

- Now, run LDA again using this new matrix

In [ ]:
lda = gensim.models.LdaModel(corpus_filtered, id2word=id2word_filtered, num_topics=6)
lda.print_topics()

- We can also use this model to get topic probabilities for unseen documents:

In [ ]:
unseen_doc = "I went to the baseball game and say the player hit a homerun !"
unseen_doc_bow = [word2id_filtered.get(word.lower(),-1) for word in unseen_doc.split()]
unseen_doc_vec = np.zeros(len(word2id_filtered))
for word in unseen_doc_bow:
    if word >= 0:
        unseen_doc_vec[word] += 1
unseen_doc_vec = unseen_doc_vec[np.newaxis]
unseen_doc_corpus = gensim.matutils.Dense2Corpus(unseen_doc_vec, documents_columns=False)
vector = lda[unseen_doc_corpus]  # get topic probability distribution for a document
for item in vector:
    print(item)

- pyLDAvis is a nice tool for visualizing our topics:

In [ ]:
pyLDAvis.enable_notebook()

# need to create a gensim dictionary object instead of our
# lightweight dict object - this is what pyLDA expects as input
dictionary = gensim.corpora.Dictionary()
dictionary.token2id = word2id_filtered

# visualize the LDA model
vis = pyLDAvis.gensim_models.prepare(lda, corpus_filtered, dictionary)
vis

### Lexical Resources

- For a fast and easy content analysis, use one of the many available prebuilt dictionaries/lexicons.
    - These map words or stems to semantic categories.
- We have discussed several lexicons in the slides.
- As an example, let's load the lexicon for measuring personal values in text:

In [ ]:
def load_lexicon(lexicon_file_path):
    word2cat = collections.defaultdict(list)
    with open(lexicon_file_path,'r') as lexicon_file:
        for line in lexicon_file:
            if line:
                word, cat = line.strip().split(", ")
                word2cat[word].append(cat)
    return word2cat
            
values_lexicon = load_lexicon("values_lexicon.txt")
print("Loaded lexicon with",len(values_lexicon),"entries.")
print("The categories for 'mother' are:",values_lexicon['mother'])

It's very easy to score a document for each category:

In [ ]:
file_list = ["christian_500.txt", "business_500.txt", "college_500.txt"]

for file_name in file_list:
    category_counts = collections.defaultdict(int)
    
    # just look at the first 25K characters
    # this way, we don't need to normalize based on the length of the document
    # and we'll save some time since this is just for demonstration purposes
    text = open(file_name).read().lower()[:25000]
    for pattern, categories in values_lexicon.items():
        count = re.findall(r'\b' + pattern + r'\b', text)
        if count:
            for category in categories:
                category_counts[category] += len(count)
    print(file_name,sorted(category_counts.items(),key=lambda x:x[1], reverse=True))

---

## A - Word Embeddings

In [ ]:
#@title Setup (click the "run" button to the left) {display-mode: "form"}

## Setup ##

# imports

# built-in Python libraries
# -------------------------

# for defaultdict data structure
import collections
# for reading csv files
import csv
# operating system functions
import os

# 3rd party libraries
# -------------------

import numpy as np

! pip install --upgrade gensim
import gensim
import gensim.test.utils
import gensim.scripts.glove2word2vec
import gensim.models.fasttext

import scipy.spatial.distance

import nltk
nltk.download('stopwords')

! pip install fasttext
import fasttext

if not os.path.exists("glove.twitter.27B.zip"):
    !wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
else:
    print("GloVe already downloaded.")
if not os.path.exists("glove.twitter.27B.50d.txt"):
    !unzip glove.twitter.27B.zip
else:
    print("GloVe already extracted.")
if not os.path.exists("questions-words.txt"):
    !wget http://download.tensorflow.org/data/questions-words.txt
else:
    print("Word analogies data already loaded.")
if not os.path.exists("crawl-300d-2M-subword.zip"):
    !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
else:
    print("Fasttext embeddings already downloaded")
if not os.path.exists("crawl-300d-2M-subword.vec"):
    print("Extracting Fasttext embeddings. This may take several minutes...")
    !unzip crawl-300d-2M-subword.zip
else:
    print("Fasttext already extracted.")
if not os.path.exists("Stsbenchmark.tar.gz"):
    !wget http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
    !tar -xzf Stsbenchmark.tar.gz
print()
print("Done with setup!")
print("If you'd like, you can click the (X) button to the left to clear this output.")

### Using gensim for word embeddings

- Gensim is back: this time we'll use it to experiment with word embeddings.
- We can use it to load an embeddings matrix in several formats.
    - Since we've been working with some social media data, let's load the GloVe Twitter embeddings:

In [ ]:
# Load 100-dimensional vectors
glove_file_path = "glove.twitter.27B.100d.txt"
# First convert to word2vec format (what gensim expects)
tmp_file_path = gensim.test.utils.get_tmpfile("word2vec_twitter_100d.txt")
_ = gensim.scripts.glove2word2vec.glove2word2vec(glove_file_path, tmp_file_path)

# Now load with gensim (may take several minutes)
print("Loading GloVe...")
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file_path)
print("GloVe was loaded into memory.")

- **Now, what can we do with these embeddings?**
- Find the most similar words to a query:

In [ ]:
glove.most_similar("france")

- Measure similarity between any pair of words:

In [ ]:
pairs = [ ('data','pastry') , ('beautiful','lovely'), ('hot','cold'), \
          ('a person walked their dog','a dog walked with a person') , \
          ('the school was prestigious','the university was highly ranked')]

for pair in pairs:
    sim = glove.n_similarity(pair[0].split(),pair[1].split())
    print(pair,":",sim)
print()

- Or just get the vector representation for a word:

In [ ]:
vector = glove['summer']
print(vector)

### Word embedding geometry

- The classic example:

In [ ]:
mystery_word_vec = glove['king'] - glove['man'] + glove['woman']
glove.similar_by_vector(mystery_word_vec)

### Putting it together: word analogies

- In the not-too-distance past, these types of questions appeared on American college entrance examinations:
    - e.g., "man:king::woman:?"
        - answer: queen
- Around 2013, when word vectors were becoming extremely popular, this task was studied within the NLP community, and along with it came some standard datasets.
    Let's load one of them here:

In [ ]:
analogies = collections.defaultdict(list)
category = ""
with open("questions-words.txt",'r') as analogy_questions:
    for aq in analogy_questions:
        if aq.startswith(":"):
            category = aq.split(":")[1].strip()
        else:
            a,b,c,d = aq.split()
            analogies[category].append([a,b,c,d])
            
print("Loaded analogies with",len(analogies),"categories.")

**Exercise 5**: Word analogies
- Using the GloVe embeddings that we loaded before, write a function to complete word analogies.
    - return a list of the top `top_n` guesses, in order from most likely to least likely.

In [ ]:
# solve an analogy a is to b as c is to ?
def guess_analogy(a,b,c,vectors,top_n=5):
    
# ------------- Exercise 6 -------------- #


    return []
# ---------------- End ------------------ #
    

# quick test
a,b,c,d = "man","king","woman","queen"
guesses = guess_analogy(a,b,c,glove)
if d in guesses:
    print("queen was in the top n guesses!")
else:
    print("queen was NOT in the top n guesses...")
    print("guesses were:",guesses)

In [ ]:
#@title Sample Solution (double-click to view) {display-mode: "form"}

def guess_analogy(a,b,c,vectors,top_n=5):
    
# ------------- Exercise 6 -------------- #

    inputs = set([a,b,c])
    guess_vector = vectors[b] - vectors[a] + vectors[c]
    guesses = glove.similar_by_vector(guess_vector)
    return [item[0] for item in guesses if item[0] not in inputs][:top_n]

# ---------------- End ------------------ #


# quick test
a,b,c,d = "man","king","woman","queen"
guesses = guess_analogy(a,b,c,glove)
if d in guesses:
    print("queen was in the top n guesses!")
else:
    print("queen was NOT in the top n guesses...")
    print("guesses were:",guesses)

- When you are ready, run your function on some more of the dataset:

In [ ]:
# evaluation

# just check 100 per category, otherwise you will have to wait a while
max_per_category = 100
correct = collections.defaultdict(int)
top_5_correct = collections.defaultdict(int)
total = collections.defaultdict(int)
for category, analogy_questions in analogies.items():
    print("evaluating",category)
    for aq in analogy_questions[:max_per_category]:
        a,b,c,d = aq
        if all([item in glove for item in [a,b,c,d]]):
            guesses = guess_analogy(a,b,c,glove)
            total[category] += 1
            if d in guesses:
                top_5_correct[category] += 1
                if d == guesses[0]:
                    correct[category] += 1
                
global_correct = 0
global_top_5_correct = 0
global_total = 0
print()
print("Category-level results")
for category in analogies:
    if total[category]:
        print(category)
        print("\t","top-1 score:",float(correct[category])/total[category])
        print("\t","top-5 score:",float(top_5_correct[category])/total[category])
        global_correct += correct[category]
        global_top_5_correct += top_5_correct[category]
        global_total += total[category]
print("Overall results")
print("\t","top-1 score:",float(global_correct)/global_total)
print("\t","top-5 score:",float(global_top_5_correct)/global_total)

- Did you get better results than the sample solution?
    - It achieved, overall:
        - top-1: 0.46875
        - top-5: 0.640625
- What could you do to improve the results?

---
## Sub-words and Compositionality

- You may have noticed that our word embeddings don't always work:

In [ ]:
# note that the current version of GloVe Twitter is about 5 years old now
try:
    print(glove['adulting'])
except Exception as e:
    print("Error:",e)

- How could we address this?
- We could:
    - skip words for which we do not have embeddings
    - use the same "out of vocabulary" (OOV) vector to represent each unknown word.
    - re-train our word embeddings every...
        - year? month? day?
    - change the unit of semantics from *words* to *something else*.
        - What else might we choose?
            - Some proposals include subwords, word pieces, and characters.
- Let's consider the last approach, first by looking at subword embeddings with fasttext:

### Sub-words embeddings with FastText

In [ ]:
# we can also load these using gensim
# this will also take several minutes to load into memory
emb_path = "crawl-300d-2M-subword.bin"
fasttext_model = fasttext.load_model(emb_path)

- We can do the same kinds of things that we did before with GloVe:

In [ ]:
print("First 50 dimensions of vector for computer:")
print(fasttext_model['computer'][:50])

print("Similarity between 'forest' and 'trees':")
print(1- scipy.spatial.distance.cosine(fasttext_model['forest'], fasttext_model['trees']))

- In addition, we can use subword information to reason about unknown words:

In [ ]:
print("First 50 dimensions of vector for adulting:")
print(fasttext_model['adulting'][:50])
print()
# just to prove that totally new words are handled
print("First 50 dimensions of vector for howhoozaling:")
print(fasttext_model['howhoozaling'][:50])

### Semantic compositions

- Simply averaging word embeddings (mean pooling) turns out to be a strong baseline for short text representations.

In [ ]:
sent1 = "The airplane flew over the fields"
sent2 = "A train crossed the river"

sent1_emb = np.mean([fasttext_model[w.lower()] for w in sent1.split()],axis=0)
sent2_emb = np.mean([fasttext_model[w.lower()] for w in sent2.split()],axis=0)

print("Similarity:",1- scipy.spatial.distance.cosine(sent1_emb,sent2_emb))

### Putting it together: short text similarity

- SemEval is a yearly competition to solve a range of semantic NLP tasks.
- A common SemEval task is "semantic text similarity".
    - The goal is to build a model that can produce similarity scores for pairs of texts that are highly correlated with human judgements of similarity.
    - Let's load some data for this task:

In [ ]:
test_sts_data = []
fnames = ['section1','section2','section3','docid','score','doc1','doc2']
with open("stsbenchmark/sts-test.csv",'r') as infile:
    reader = csv.DictReader(infile, fieldnames=fnames,dialect=csv.excel_tab)
    for row in reader:
        test_sts_data.append(row)
print("Loaded",len(test_sts_data),"test pairs.")
print("Example:",test_sts_data[0])

- Now, let's build a simple system to get some reasonable results on this task.
    - hint: stopword removal should be helpful here!

In [ ]:
def get_text_sim_score(text1,text2,word_embeddings):

# ------------- Exercise 3 -------------- #



# ---------------- End ------------------ #
    return score

In [ ]:
#@title Sample Solution (double-click to view) {display-mode: "form"}
stopwords = set(nltk.corpus.stopwords.words('english'))

def get_text_sim_score(text1,text2,word_embeddings):

# ------------- Exercise 3 -------------- #

    vec1 = np.mean([word_embeddings[w] for w in text1.split() if w.lower() not in stopwords],axis=0)
    vec2 = np.mean([word_embeddings[w] for w in text2.split() if w.lower() not in stopwords],axis=0)
    score = 1 - scipy.spatial.distance.cosine(vec1,vec2)

# ---------------- End ------------------ #
    return score

- Let's evaluate the performance:

In [ ]:
golds, preds = [],[]
for test_data in test_sts_data:
    text1 = test_data['doc1']
    text2 = test_data['doc2']
    if text1 and text2:
        gold = float(test_data['score'])
        pred = get_text_sim_score(text1,text2,fasttext_model)
        golds.append(gold)
        preds.append(pred)

print("Some examples of pairs and their human annotated scores:")
for item in test_sts_data[:5]:
    print(item['score'],item['doc1'],item['doc2'])
print("Correct labels:",golds[:5])
print("Predictions:",preds[:5])
print("Correlation Score (rho, p-value):",scipy.stats.pearsonr(golds,preds))

- That is a good start! 
- It's definitely possible to do better with different compisition functions/networks, or even this same approach with different embeddings.
    - See some of the state-of-the-art results here: http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark